## Duckbot Automated Growth assays code library

### READ ME
1. Define the precise plate and well positions for your machine in Plate_positions.py
2. Define the dimensions of the labwware you will use for media dispensing under "Define labware and tools"
3. Run all cells in this  script in order and then 
Run these cells to import libraries and define methods

In [26]:
import random
import pandas as pd
import DuckbotExptSetupUtils as exp
import os
import json
import Plate_positions as pp # <--- USER NOTE: Absolute positions of plates and wells are defined in this python script. 

In [ ]:
def divide_chunks(l, n):
    for i in range(0, len(l), n):    # looping till length l
        yield l[i:i + n]
        
def dispense_to_wells(m, well_coords, z, dispense_vol,dispenses_per_syringe_fill, media_reservoir):
    dispense_chunks = list(divide_chunks(well_coords, dispenses_per_syringe_fill))
    for wells in dispense_chunks:
#         m.moveTo(x=media_reservoir["x"], y=media_reservoir['y'])
#         m.moveTo(z=media_reservoir["aspiration_position"]
#         m.move(de=dispense_vol * dispenses_per_syringe_fill)
#         m.move(dz=media_reservoir ["wall_height"])
        for well in wells:
            print("dispensing")
#             m.moveTo(x=well_coords[0], y=well_coords[1])
#             m.moveTo(z=z)
#             m.move(de=-dispense_vol)
        

    
def add_well_coords_to_df(plate_num, df):
    well_coord_list_of_dicts = pp.fetch_plate_wellpostions(plate_num)
    print(well_coord_list_of_dicts)
#     plate_df = df.loc[df['Plate'] == f'Plate_{plate_num}']
    for index, row in df.iterrows():
        print(row['Well'])
        for well in well_coord_list_of_dicts:
            if row['Plate'] == f'Plate_{plate_num}' and row['Well'] == well['well_id']:
                df.loc[index, 'x'] = well['x']
                df.loc[index, 'y'] = well['y']
    


## 1. Define labware and tools
### Labware config note
All should be in arbitrary Jubilee motor units. Find this manually on the machine. They shouldn't need to be updated until you start using a new reservoir. And if that's the case you could create a labware library and then define the media_reservoir variable as one of the objects in your library. 

In [2]:
media_reservoir = { #
'x' : 50, 'y' : 50, #X and Y should get the machine to the center of the reservoir. 
'aspiration_position': 50, #Z-axis point that is just above the bottom of the reservoir.  
'wall_height' : 50 #Relative Z-axis units from the aspiration position
}
well_z = 10 #Distance to move down into the well for dispensing


#Tool positions
media_syringe = 2 #What jubilee tool position did you define this as
duckweed_syringe = 3
dispenses_per_syringe_fill = 20
dispense_vol = 10 


In [3]:
#Establish machine communication 
port = '/dev/ttyACM0' #Check options using  - print([port.name for port in serial.tools.list_ports.comports()])
m = MachineCommunication(port)

NameError: name 'MachineCommunication' is not defined

### Create dataframe with experiment metadata

In [4]:
# USER ACTION REQUIRED

#1. DEFINE EXPERIMENTAL VARIABLES
genotypes = ["Sp7498", "sp3484"] # Replace with names for unique duckweed genotypes
media = ["Mock", "Salt"] # Replace with names for unique media
reps  = 3 # Replace with your desired number of replicates for each duckweed/media combination. 

#2. DEFINE FILE LOCATION AND NAME
expt_setup_parent_dir = os.getcwd() # Default uses current working directory but you can replace with your own choice. 
expt_setup_dir = "TestExptDir" # Name of the folder to hold experiment data and metadata including the setup file
expt_setup_file_name = "TestFile.json" #Name for the experiment setup file (Metadata)

        

In [5]:
# Creates master list of sample info, shuffles and then assigns to plates and wells. 
master_expt_list = []

for g in genotypes:
    for m in media:
        for x in range(reps):
             master_expt_list.append({"genotype": g, "media": m, "condition_replicate": x + 1})


random.shuffle(master_expt_list)
master_expt_list = exp.assign_plates_and_wells(master_expt_list)
expt_dict = {"sample_info" : master_expt_list}




In [6]:
#Create folder and save experimental set up file to it

path = os.path.join(expt_setup_parent_dir, expt_setup_dir)

os.mkdir(path)
os.chdir(path)

with open(expt_setup_file_name, 'w') as f:
    json.dump(expt_dict, f)

FileExistsError: [Errno 17] File exists: '/Users/Orlando/Documents/Github_clones/duckbot/notebooks/TestExptDir'

In [10]:
path = os.path.join(expt_setup_parent_dir, expt_setup_dir)

os.chdir(path)
print(os.getcwd())

/Users/Orlando/Documents/Github_clones/duckbot/notebooks/TestExptDir


In [11]:
# Turn samples list into a dataframe

#Import from file (in case user wants to make any manual edits to the JSON file after creating it)

#TO DO import from Json
with open(expt_setup_file_name) as datafile:
    expt_data = json.load(datafile)

sample_data = expt_data["sample_info"]
df = pd.DataFrame(sample_data)
print (df[0:])


   genotype media  condition_replicate    plate_well_id    Plate Well
0    Sp7498  Salt                    3  Plate_1_Well_A1  Plate_1   A1
1    Sp7498  Salt                    2  Plate_1_Well_A2  Plate_1   A2
2    sp3484  Mock                    2  Plate_1_Well_A3  Plate_1   A3
3    sp3484  Salt                    2  Plate_1_Well_A4  Plate_1   A4
4    sp3484  Salt                    1  Plate_1_Well_A5  Plate_1   A5
5    Sp7498  Mock                    3  Plate_1_Well_A6  Plate_1   A6
6    sp3484  Salt                    3  Plate_1_Well_B1  Plate_1   B1
7    sp3484  Mock                    1  Plate_1_Well_B2  Plate_1   B2
8    sp3484  Mock                    3  Plate_1_Well_B3  Plate_1   B3
9    Sp7498  Mock                    1  Plate_1_Well_B4  Plate_1   B4
10   Sp7498  Mock                    2  Plate_1_Well_B5  Plate_1   B5
11   Sp7498  Salt                    1  Plate_1_Well_B6  Plate_1   B6


### Set up physical plates

#### Label Plates and add plates to machine

In [12]:
num_plates = df.Plate.nunique()
print(num_plates)

print("This experiment requires {} 24-well plate(s)".format(num_plates))
print("----")
lst = list(range(1,num_plates + 1))
for n in lst:
    print("Label a plate with experiment ID or initials and 'plate {}'".format(n))
print("----")
print ("Place the 24-well plate(s) in the jubilee".format(num_plates))
print ("Start at position 1 and fill empty plate slots in order")



1
This experiment requires 1 24-well plate(s)
----
Label a plate with experiment ID or initials and 'plate 1'
----
Place the 24-well plate(s) in the jubilee
Start at position 1 and fill empty plate slots in order


#### Add media to plates
When prompted insert containers of the relevant sterile media into the input slot on the Jubilee. 

In [15]:
#Retrieve absolute positions of wells from a library and then add those coordinatest to the plate set up dataframe

num_plates = df.Plate.nunique()
plates = list(range(1, num_plates + 1))
print(plates)

for p in plates:
    add_well_coords_to_df(p, df)

print(df)

[1]


In [24]:
#Reorganize dataframe to create machine instructions sorted by media-type

media_df = df.groupby(['media']) #Returns a list of tuples with [0] being the group key and [1] the dataframe
media_dicts = []

for media in media_df:
    well_coords = []
    for index, row in media[1].iterrows():
        this_well = []
        this_well.append(row['x'])
        this_well.append(row['y'])
        well_coords.append(this_well)
    media_dicts.append({'media': media[0], 'well-coords' : well_coords})
print(media_dicts)


[{'media': 'Mock', 'well-coords': [[44.0, 210.0], [95.0, 210.0], [27.0, 193.0], [44.0, 193.0], [61.0, 193.0], [78.0, 193.0]]}, {'media': 'Salt', 'well-coords': [[10.0, 210.0], [27.0, 210.0], [61.0, 210.0], [78.0, 210.0], [10.0, 210.0], [95.0, 193.0]]}]


In [25]:
# Send machine instructions
for media in media_dicts:
     print(f"Please ensure {media['media']} is available in the machine before continuing.")
     while True:
        value = input("Enter 'YES' to confirm that the correct media is in position")
        if value != "YES":
            print("Please confirm")
        else:
            break
     dispense_to_wells(m, media["well-coords"], well_z, dispenses_per_syringe_fill, dispense_vol, media_reservoir)

Please ensure Mock is available in the machine before continuing.
Enter 'YES' to confirm that the correct media is in positionYES
dispensing
dispensing
dispensing
dispensing
dispensing
dispensing
Please ensure Salt is available in the machine before continuing.
Enter 'YES' to confirm that the correct media is in positionYES
dispensing
dispensing
dispensing
dispensing
dispensing
dispensing


In [1]:
# media_df = df.groupby('media')
# for media_type, sample_df in media_df:
#     print("Place container of media type **{0}** into jubilee and ensure lid is open".format(media_type))
#     print("""Type anything into the input field to confirm that the media is available.
#     After this point the Jubilee will begin dispensing""")
#     input() 
#     for index,s in sample_df.iterrows():
#         #move to plate and well
#         print("Move to {0}, well {1}".format(s["Plate"], s["Well"]))
#         print("Dispensing media of type {0} into {1}, well {2}".format(media_type,s["Plate"], s["Well"]))
#     print("---")
    
    



In [ ]:
m.toolChange(2)
m.setExtruderRelative()
for 

#### Add duckweed to wells
Place a container filled with fronds of the relevant duckweed type and the machine will attempt to move individual fronds into the relevant wells. After each attempt at filling all wells a camera will take pictures of each well to confirm success and then unsuccessful wells will be reattempted

In [10]:
grouped_df = df.groupby('genotype')
for field_value, sample_df in grouped_df:
    print("Place container of duckweed type **{0}** into jubilee and ensure lid is open".format(field_value))
    print("""Type anything into the input field to confirm that the media is available.
    After this point the Jubilee will begin dispensing""")
    input() 
    for index,s in sample_df.iterrows():
        #move to plate and well
        print("Move to {0}, well {1}".format(s["Plate"], s["Well"]))
        print("Dispensing media of type {0} into {1}, well {2}".format(field_value,s["Plate"], s["Well"]))


Place container of duckweed type **Sp7498** into jubilee and ensure lid is open
Type anything into the input field to confirm that the media is available.
    After this point the Jubilee will begin dispensing
ok
Move to Plate_1, well A2
Dispensing media of type Sp7498 into Plate_1, well A2
Move to Plate_1, well A3
Dispensing media of type Sp7498 into Plate_1, well A3
Move to Plate_1, well A4
Dispensing media of type Sp7498 into Plate_1, well A4
Move to Plate_1, well B1
Dispensing media of type Sp7498 into Plate_1, well B1
Move to Plate_1, well B2
Dispensing media of type Sp7498 into Plate_1, well B2
Move to Plate_1, well B5
Dispensing media of type Sp7498 into Plate_1, well B5
Place container of duckweed type **sp3484** into jubilee and ensure lid is open
Type anything into the input field to confirm that the media is available.
    After this point the Jubilee will begin dispensing
ok
Move to Plate_1, well A1
Dispensing media of type sp3484 into Plate_1, well A1
Move to Plate_1, well